In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)

# Quickstart 2 - Power Flow

## Problem Description

Consider a three-zone electricity market with the following loads and generators:

| Zone | Load (MW) | Generator | Capacity (MW) | Marginal Cost (€/MWh) |
|------|-----------|-----------|---------------|------------------------|
| 1    | 50        | Gen A     | 140           | 7.5                    |
|      |           | Gen B     | 285           | 6.0                    |
| 2    | 60        | Gen C     | 90            | 14.0                   |
| 3    | 300       | Gen D     | 85            | 10.0                   |

The zones are connected by transmission lines running on 11 kV with the following characteristics:

| Line | From | To | Capacity (MW) | Reactance (Ohm) |
|------|------|------|--------------|----------------|
| 1 | Zone 1 | Zone 2 | 126 | 0.2 |
| 2 | Zone 1 | Zone 3 | 250 | 0.2 |
| 3 | Zone 2 | Zone 3 | 130 | 0.1 |

Find the least-cost dispatch of the generators to meet the loads while respecting the transmission capacities. Identify the marginal prices at each bus and the flow on the transmission lines.
Calculate the non-linear power flow based on the optimised results, and determine the losses on each line as well as the voltage angles at each bus.

## PyPSA Solution

Start by importing PyPSA, creating a new network instance and adding the three buses with voltage level 11 kV:

In [ ]:
from numpy import pi

import pypsa

n = pypsa.Network()

n.add("Bus", ["zone_1", "zone_2", "zone_3"], v_nom=11)

Adding multiple components with `n.add()` is supported by passing lists for each argument. Any scalar values are broadcasted to all components.

Thus, we can add the buses, loads, generators, and lines in one go and solve the network::

In [ ]:
n.add(
    "Load",
    ["load_1", "load_2", "load_3"],
    bus=["zone_1", "zone_2", "zone_3"],
    p_set=[50, 60, 300],
)

n.add(
    "Generator",
    ["gen_A", "gen_B", "gen_C", "gen_D"],
    bus=["zone_1", "zone_1", "zone_2", "zone_3"],
    p_nom=[140, 285, 90, 85],
    marginal_cost=[7.5, 6, 14, 10],
)

n.add(
    "Line",
    ["line_1", "line_2", "line_3"],
    bus0=["zone_1", "zone_1", "zone_2"],
    bus1=["zone_2", "zone_3", "zone_3"],
    s_nom=[126, 250, 130],
    x=[0.02, 0.02, 0.01],
    r=0.01,
)

n.optimize(log_to_console=False)

It is possible to inspect the underlying `linopy` model once either `n.optimize()` or `n.optimize.create_model()` have been called. This can be done by accessing the `n.model` attribute, which prints the variable and constraint names and their dimensions.

In [ ]:
n.model

To look at the equations of a specific constraint, such as the nodal balances of the buses in each time step, execute:

In [ ]:
n.model.constraints["Bus-nodal_balance"]

As in the first example, the optimal solution can be accessed from the network object:

In [ ]:
display(n.buses_t.marginal_price)
display(n.generators_t.p)
display(n.lines_t.p0)

For the next step, we want to calculate the non-linear AC power flow based on the optimised results (which uses the linearised DC power flow model) using the Newton-Raphson method.
This can be done by calling [`n.pf()`](), but first we need to provide the set points for the generator dispatch.
These will be kept in the power flow, except for the slack generator, the output of which will be adjusted to balance the network.

In [ ]:
n.optimize.fix_optimal_dispatch()

display(n.generators_t.p_set)
display(n.generators.control)

n.pf()

Alright, it converged! Now we can see how the slack generator is used to balance
the line losses as well as the reactive power of the generators and lines. The
linear power flow necglects both.


In [ ]:
display(n.generators_t.p)
display(n.generators_t.q)
display(n.lines_t.q0)
display((n.lines_t.p0 + n.lines_t.p1).sum().sum())  # active power losses

You can see that the increase in the slack generator's output is equal to the sum of the active power losses in the lines.

With non-linear power flow, the voltage angles at the buses are calculated as well, which are stored in radians but can easily be converted to degrees:

In [ ]:
n.buses_t.v_ang * 180 / pi

This example is based on Tom Brown's [Energy Systems](https://nworbmot.org/courses/es-25) course, taken from the lecture [Complex Markets](https://nworbmot.org/courses/es-25/es-9-complex_markets.pdf), slides 42ff.